# SageMaker TorchPoints3d

This notebook is for creating Amazon SageMaker training and inference containers for PyTorch Points 3D. 

Getting Started:

1. Clone the PyTorch Points 3D [GitHub repoistory](https://github.com/nicolas-chaulet/torch-points3d) and download this notebook into the working directory.

```
$ git clone https://github.com/nicolas-chaulet/torch-points3d
```

`NOTE`: This code is under active development, tested July 22 commit `12dfca3e94add3981f7f37db25df1e5acee640fe`

This notebook will take you through the following steps:

1. Build and register Training Containers
2. Build and register Inference Container
3. Train model
4. Run Inference
5. Visualize results


## Download Dataset

Download the `shapenet` dataset, should take about 1 minute.

In [ ]:
!wget --no-check-certificate "https://shapenet.cs.stanford.edu/media/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip"

Upzip the dataset, should take about 20s, size should be 2.8GB

In [ ]:
%%time

!mkdir -p dataset/raw
!unzip -q shapenetcore_partanno_segmentation_benchmark_v0_normal.zip -d ./dataset

In [ ]:
!du -h dataset

Upload the torchpoints3d dataset to which you upload the `shapnet/raw` folder, should take about 3mins to upload 16k files.

In [ ]:
from sagemaker import get_execution_role
from sagemaker import session

s3_shapenet_uri = 's3://{}/torchpoints3d'.format(session.Session().default_bucket())

In [ ]:
%%time

!aws s3 sync dataset/shapenetcore_partanno_segmentation_benchmark_v0_normal $s3_shapenet_uri/shapenet/raw --quiet

Check that we have some files uploaded

In [ ]:
!aws s3 ls $s3_shapenet_uri/shapenet/raw/

### Test training container

You can test the training container by running an interactive docker container and attaching the downloaded dataset

```
$ docker run -it --mount src="$(pwd)/dataset/shapenetcore_partanno_segmentation_benchmark_v0_normal",target="/opt/ml/input/data/training/shapenet/raw",type=bind sagemaker-torchpoints3d-training:latest
```

Once in the container, create the output directory and run the training script

```
$ cd /opt/ml/code
$ mkdir -p /opt/ml/model /opt/ml/output/data
$ SM_MODEL_DIR=/opt/ml/model SM_OUTPUT_DATA_DIR=/opt/ml/output/data SM_CHANNEL_TRAINING=/opt/ml/input/data/training python sagemaker_train.py --epochs 3
```

This will run for a few short epochs, and write the output model to `/opt/ml/model` and training logs to `/opt/ml/output/data`

Delete the zip and dataset folder

In [ ]:
!rm shapenetcore_partanno_segmentation_benchmark_v0_normal.zip && rm -Rf dataset

## Train 3d Point cloud

Train a 3d point cloud for the `shapenet` dataset

`local_estimator.fit()` will run the training job on the jupyter notebook and `estimator.fit()`

In [ ]:
import boto3
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

account_id = boto3.client('sts').get_caller_identity().get('Account')
region =  boto3.session.Session().region_name
role = get_execution_role()

training_image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-torchpoints3d-training:latest'.format(account_id, region)

hyperparameters = {"epochs": 100,
                   "lr": 0.01}

estimator = Estimator(training_image,
                      role=role,
                      train_instance_count=1,
                      train_instance_type='ml.p3.2xlarge',
                      image_name=training_image,
                      hyperparameters=hyperparameters)

estimator.fit(s3_shapenet_uri)

Download the training job model archive and list the contents

In [ ]:
!aws s3 cp $estimator.model_data .
!mkdir -p model && tar -xvf model.tar.gz -C model

## Inference

Download a sample input file

In [ ]:
!aws s3 cp $s3_shapenet_uri/shapenet/raw/02691156/1021a0914a7207aff927ed529ad90a11.txt test_inf.txt

Inspect the file

In [ ]:
!head test_inf.txt

### Test Inference Container

You can test the inference container by running an interactive docker container on port 8080 and attaching the trained model.

```
$ docker run --rm -p 8080:8080 --mount src="$(pwd)/model",target="/opt/ml/model",type=bind sagemaker-torchpoints3d-inference:latest serve
```

Once your model is running you can, check the ping response:

```
$ curl localhost:8080/ping
{
  "status": "Healthy"
}
```

Then post a request to the invocation endpoint with the sample file

```
$ curl -X POST http://localhost:9090/predictions/model -T test_inf.txt
```


## Deploy model

Deploy the model for real-time inference

In [ ]:
from time import gmtime, strftime

sm_client = boto3.client(service_name='sagemaker')

### Import model into hosting

First we create a model with the inference container

In [ ]:
from sagemaker.utils import name_from_image

inference_image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-torchpoints3d-inference:latest'.format(account_id, region)

# Get a endpoint name based on the image
endpoint_name = name_from_image(inference_image)

container = {
    'Image': inference_image,
    'ModelDataUrl': estimator.model_data
}

create_model_response = sm_client.create_model(
    ModelName = endpoint_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

### Create endpoint configuration

In [ ]:
endpoint_config_name = endpoint_name + 'EPConf'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.c5.xlarge',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': endpoint_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

## Create endpoint

In [ ]:
import time

endpoint_name = endpoint_name
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Load the input files into a byte array

In [ ]:
filename = 'test_inf.txt'

with open(filename, 'rb') as file:
    body = file.read()
    body = bytearray(body)

Perform inference with the boto3 client

In [ ]:
%%time 

import boto3

client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= body,
    ContentType = 'application/octet-stream')

results = response['Body'].read()
len(results)

## Visualize

Load the results from prediction as numpy array and visualize with [mplot3d](https://matplotlib.org/mpl_toolkits/mplot3d/index.html) or iteratively in jupyter lab with [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/install.html#for-jupyter-lab-users)

In [ ]:
!pip install ipyvolume -q

In [ ]:
import numpy as np
import json

data = np.array(json.loads(results)['response'])
print(data.shape)

data[0:2]

Visualize with mplot3d

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

x,y,z,c = data[:,0], data[:,1], data[:,2], data[:,3]

ax.scatter(x, y, z, c=c, marker='o')

ax.set(xlim=(-0.4, 0.4), ylim=(-0.4, 0.4))
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.view_init(elev=0., azim=90)

plt.show()

Visualize point cloud in 3d ipvolume widget

In [ ]:
import ipyvolume as ipv

def get_coord(data, color):
    mask = data[:,3]==color
    return data[:,0][mask], data[:,1][mask], data[:,2][mask]

fig = ipv.figure(width=600, height=600)

x,y,z = get_coord(data, 6)
scatter = ipv.scatter(x, y, z, size=1, marker='sphere', color='grey')

x,y,z = get_coord(data, 7)
scatter = ipv.scatter(x, y, z, size=1, marker='sphere', color='yellow')

ipv.xyzlim(-0.5, 0.5)
ipv.show()